In [9]:
import sys
print(sys.executable)
# Verify kernal path
import numpy as np
import pandas as pd
import os
import requests
from dotenv import load_dotenv
load_dotenv()

tabular_prices = pd.read_parquet("../data/transformed/tabular_prices_2022-08-02-2024-08-02.parquet")
tabular_prices


/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


,0,1,2,3,4,5,6,7,8,9,...,212,213,214,215,216,217,218,219,220,target_close_price_next_day
0,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659398e+18,0.000000,56941340.0,...,0.363333,63496620.0,-0.073343,171.024002,169.102997,170.947968,167.599106,2.005173,0.002107,173.190002
1,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659485e+18,0.000000,56941340.0,...,0.363333,60280852.0,-0.079148,171.813995,169.102997,171.695312,168.131577,1.902056,0.002102,173.029999
2,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659571e+18,0.000000,56941340.0,...,-0.160000,60127840.0,0.042250,172.722000,169.102997,172.140213,168.598099,0.441202,-0.000924,174.550003
3,56696984.0,164.743195,163.210007,165.350006,165.850006,163.000000,491310.00000,1.659672e+18,0.000000,56941340.0,...,1.520000,62428372.0,0.410894,173.212006,169.102997,172.943466,169.164948,0.795744,0.008785,174.149994
4,57918768.0,165.126770,164.263336,165.190002,166.503326,163.399994,507545.65625,1.659744e+18,-0.160000,56941340.0,...,-0.400000,62208356.0,-0.216891,173.549332,169.102997,173.345642,169.639709,0.755515,-0.002292,171.520004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,38468544.0,234.299194,235.000000,234.820007,236.270004,232.330002,623906.00000,1.721102e+18,0.420000,52547780.0,...,0.093333,44989644.0,-0.040710,218.037994,225.936493,218.969620,222.390121,0.377636,0.000428,218.240005
715,55878904.0,228.839600,229.449997,228.880005,231.459900,226.639999,811024.00000,1.721189e+18,-5.940000,53399272.0,...,0.093333,40082904.0,-0.042438,217.977997,225.414505,218.726410,221.994873,0.292077,0.000428,218.800003
716,64346032.0,225.181000,230.279999,224.179993,230.440002,222.270004,844538.00000,1.721275e+18,-4.700000,55351372.0,...,0.560000,38128876.0,0.157249,218.240005,224.705505,218.750946,221.690598,0.329983,0.002566,222.080002
717,48020040.0,224.735596,224.820007,224.309998,226.800003,223.274994,612739.00000,1.721362e+18,0.130000,53445452.0,...,3.280000,39847940.0,0.190291,219.063995,224.431000,219.860626,221.727676,1.710839,0.014991,218.360001


In [10]:
# TRAIN TEST SPLIT
from sklearn.model_selection import train_test_split

def train_test_split_tabular(tabular_prices, target_column_name="target_close_price_next_day", test_size=0.2, random_state=42):
    # seperate features/targets
    X = tabular_prices.drop(columns=[target_column_name])
    y = tabular_prices[target_column_name]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split_tabular(tabular_prices)
print(f"X_train: {X_train.shape}") # (examples-in-train, num-features)
print(f"y_train: {y_train.shape}") # (examples-in-train,) 1D beceause only one target
print(f"X_test: {X_test.shape}")   # (examples-in-test, num-features)
print(f"y_test: {y_test.shape}")

X_train: (575, 221)
y_train: (575,)
X_test: (144, 221)
y_test: (144,)


In [28]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

model = GradientBoostingRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 6.529209730146054


In [13]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions and calculate MSE
y_pred = rf_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Random Forest MSE: {mse}')

Random Forest MSE: 5.54467381890986


In [14]:
y_test[0:10]


120    145.470001
164    143.960007
39     150.429993
551    182.122498
199    145.910004
550    182.309998
65     143.860001
513    186.190002
78     150.649994
596    168.839996
Name: target_close_price_next_day, dtype: float32

In [15]:
y_pred[0:10]

array([143.59186752, 142.31713364, 152.68413315, 182.66161652,
       145.84414139, 184.22800034, 142.59016678, 185.21798325,
       153.19100693, 170.26291656])

In [25]:
from sklearn.neural_network import MLPRegressor
nn_model = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', max_iter=500, random_state=42)

nn_model.fit(X_train, y_train)

y_pred = nn_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 1.9796495360135436e+35


/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/lib/python3.12/site-packages/sklearn/neural_network/_stochastic_optimizers.py:275: RuntimeWarning: overflow encountered in square
  self.beta_2 * v + (1 - self.beta_2) * (grad**2)
/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/lib/python3.12/site-packages/sklearn/utils/extmath.py:205: RuntimeWarning: overflow encountered in matmul
  ret = a @ b


In [23]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor()

xgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = xgb_model.predict(X_train)

# Calculate the mean squared error
mse = mean_squared_error(y_train, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 1.842042706812208e-06


In [24]:
import lightgbm as lgb

lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)

# Train the model
lgb_model.fit(X_train, y_train)

# Predict on the test set
y_pred = lgb_model.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001664 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42363
[LightGBM] [Info] Number of data points in the train set: 575, number of used features: 221
[LightGBM] [Info] Start training from score 174.023416
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai